In [1]:
#from google.cloud import secretmanager
import pandas as pd
import numpy as np
from currensee.utils.db_utils import create_pg_engine
from sqlalchemy import text

import pandas as pd
import random
import os
import re



2025-07-18 14:09:22,921 - currensee.core.secrets - INFO - PROJECT_ID not found, attempting to load from .env file
2025-07-18 14:09:22,925 - currensee.core.secrets - INFO - Loaded environment from ./.env
2025-07-18 14:09:22,926 - currensee.core.secrets - INFO - SecretManager initialized with project_id: adsp-34002-on02-sopho-scribe


In [2]:
# Define DB_NAME
DB_NAME = "crm_outlook"

In [3]:
engine = create_pg_engine(db_name=DB_NAME)

In [4]:
pd.read_sql("SELECT * FROM preferences where email = 'jane.moneypenny@bankwell.com' order by as_of_date", con=engine)

,as_of_date,employee_first_name,employee_last_name,email,title,finance_detail,news_detail,macro_news_detail,past_meeting_detail
0,2023-01-23 10:00:00,Jane,Moneypenny,jane.moneypenny@bankwell.com,Relationship Manager,full,full,full,full
1,2023-04-02 10:00:00,Jane,Moneypenny,jane.moneypenny@bankwell.com,Relationship Manager,short,full,full,short
2,2023-07-26 14:00:00,Jane,Moneypenny,jane.moneypenny@bankwell.com,Relationship Manager,none,none,full,short
3,2023-11-04 10:00:00,Jane,Moneypenny,jane.moneypenny@bankwell.com,Relationship Manager,short,full,none,none
4,2024-01-30 10:00:00,Jane,Moneypenny,jane.moneypenny@bankwell.com,Relationship Manager,short,full,none,full
5,2024-03-10 09:00:00,Jane,Moneypenny,jane.moneypenny@bankwell.com,Relationship Manager,full,none,full,full


In [18]:
pd.read_sql("SELECT * FROM preferences where email = 'ryan.holloway@bankwell.com' order by as_of_date", con=engine)

,as_of_date,employee_first_name,employee_last_name,email,title,finance_detail,news_detail,macro_news_detail,past_meeting_detail
0,2023-01-23 10:00:00,Ryan,Holloway,ryan.holloway@bankwell.com,Relationship Manager,none,full,none,full
1,2023-04-02 10:00:00,Ryan,Holloway,ryan.holloway@bankwell.com,Relationship Manager,short,full,full,none
2,2023-07-26 14:00:00,Ryan,Holloway,ryan.holloway@bankwell.com,Relationship Manager,short,none,none,short
3,2023-11-04 10:00:00,Ryan,Holloway,ryan.holloway@bankwell.com,Relationship Manager,short,none,full,full
4,2024-01-30 10:00:00,Ryan,Holloway,ryan.holloway@bankwell.com,Relationship Manager,none,full,none,none
5,2024-03-10 09:00:00,Ryan,Holloway,ryan.holloway@bankwell.com,Relationship Manager,none,none,none,short


In [17]:
pd.read_sql("""select coalesce(as_of_date, '2023-01-23 10:00:00') as as_of_date
    from (
    SELECT max(p.as_of_date) as as_of_date
    FROM preferences p
    where p.email = 'jane.moneypenny@bankwell.com'
    and as_of_date <= '2022-04-12 10:00:00')a""", con=engine)

,as_of_date
0,2023-01-23 10:00:00


In [10]:
pd.read_sql("""SELECT as_of_date
, employee_first_name
, employee_last_name
, finance_detail
, news_detail
, macro_news_detail
, past_meeting_detail 
FROM preferences 
where employee_first_name = 'Ryan' and employee_last_name = 'Holloway' 
order by as_of_date
""", con=engine)

,as_of_date,employee_first_name,employee_last_name,finance_detail,news_detail,macro_news_detail,past_meeting_detail
0,2023-01-23 10:00:00,Ryan,Holloway,none,full,none,full
1,2023-04-02 10:00:00,Ryan,Holloway,short,full,full,none
2,2023-07-26 14:00:00,Ryan,Holloway,short,none,none,short
3,2023-11-04 10:00:00,Ryan,Holloway,short,none,full,full
4,2024-01-30 10:00:00,Ryan,Holloway,none,full,none,none
5,2024-03-10 09:00:00,Ryan,Holloway,none,none,none,short


In [9]:
pd.read_sql("""SELECT as_of_date
, employee_first_name
, employee_last_name
, finance_detail  
, news_detail
, macro_news_detail
, past_meeting_detail 
FROM preferences 
where employee_first_name = 'Jane' and employee_last_name = 'Moneypenny' 
order by as_of_date
""", con=engine)

,as_of_date,employee_first_name,employee_last_name,finance_detail,news_detail,macro_news_detail,past_meeting_detail
0,2023-01-23 10:00:00,Jane,Moneypenny,full,full,full,full
1,2023-04-02 10:00:00,Jane,Moneypenny,short,full,full,short
2,2023-07-26 14:00:00,Jane,Moneypenny,none,none,full,short
3,2023-11-04 10:00:00,Jane,Moneypenny,short,full,none,none
4,2024-01-30 10:00:00,Jane,Moneypenny,short,full,none,full
5,2024-03-10 09:00:00,Jane,Moneypenny,full,none,full,full


In [6]:
df = pd.read_sql("SELECT * from employees", con=engine)
df.head()

,index,employee_id,first_name,last_name,title,email,phone,hire_date,department,market
0,0,31e735f6-cfbe-4107-8aba-4ef45a90e9ae,Jane,Moneypenny,Relationship Manager,jane.moneypenny@bankwell.com,971.841.4711x92417,2021-01-14,Enterprise Investment,San Fransisco
1,1,b61a7a99-791a-4b9f-923f-271810defac4,Monica,Melton,Product Specialist,monica.melton@bankwell.com,884.541.7837,2021-12-03,Small Business Investment,Chicago
2,2,2f8c004d-e0a6-4bba-956e-2b60af632674,Michael,Long,Finance Assistant,michael.long@bankwell.com,478-831-1994x378,2021-12-06,Sales,Washington DC
3,3,59dcd404-9d5c-47f4-9ba1-2de5156343f8,Breanna,Guerra,Senior Relationship Manager,breanna.guerra@bankwell.com,736-597-4792x589,2024-07-11,Enterprise Investment,Boston
4,4,7a650dd8-34a3-4148-a7f8-631bf5db37d4,Heather,Singh,Financial Advisor,heather.singh@bankwell.com,7139112404,2020-11-15,Customer Support,Washington DC


In [11]:
df = pd.read_sql("""SELECT p.as_of_date
        , p.employee_first_name
        , p.employee_last_name
        , p.finance_detail
        , p.news_detail
        , p.macro_news_detail
        , p.past_meeting_detail 
        , p.email	
        FROM preferences p
        where p.email = 'jane.moneypenny@bankwell.com'
        """, con=engine)
df.head(10)

,as_of_date,employee_first_name,employee_last_name,finance_detail,news_detail,macro_news_detail,past_meeting_detail,email
0,2023-01-23 10:00:00,Jane,Moneypenny,full,full,full,full,jane.moneypenny@bankwell.com
1,2023-04-02 10:00:00,Jane,Moneypenny,short,full,full,short,jane.moneypenny@bankwell.com
2,2023-07-26 14:00:00,Jane,Moneypenny,none,none,full,short,jane.moneypenny@bankwell.com
3,2023-11-04 10:00:00,Jane,Moneypenny,short,full,none,none,jane.moneypenny@bankwell.com
4,2024-01-30 10:00:00,Jane,Moneypenny,short,full,none,full,jane.moneypenny@bankwell.com
5,2024-03-10 09:00:00,Jane,Moneypenny,full,none,full,full,jane.moneypenny@bankwell.com


In [9]:
df = pd.read_sql("""SELECT * from meeting_data where 
 host_email = 'jane.moneypenny@bankwell.com'
and invitee_emails	= 'adam.clay@compass.com'
--and meeting_timestamp > '2023-01-23 10:00:00' 

order by meeting_timestamp desc """, con=engine)
df.head(30)

,index,meeting_timestamp,host,host_email,invitees,invitee_emails,meeting_subject
0,NaN,2025-06-16 11:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - AML Requirements & Policy Developments
1,NaN,2025-06-11 14:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Tracking Error & Volatility Shift Study
2,NaN,2025-06-11 12:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Geopolitical Risks & Global Supply E...
3,NaN,2025-06-01 13:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Low-Volatility Fund Enhancements
4,NaN,2025-05-25 14:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - ESG Ratings Dashboard Review
5,NaN,2025-05-21 10:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Charitable Deduction & Gift Strategy
6,NaN,2025-05-17 16:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Strategic Risk Metrics & Tolerance R...
7,NaN,2025-05-17 16:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Capital Allocation & Milestone Mapping
8,93.0,2024-03-26 11:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Annual Credit Facility Review Meeting
9,51.0,2021-10-01 11:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Credit Facility Documentation Final ...


In [12]:
df = pd.read_sql("""SELECT * from meeting_data where 
 host_email = 'jane.moneypenny@bankwell.com'
and invitee_emails	= 'lisa.kennedy@lockheedmartincorporation.com'
and meeting_timestamp < '2023-01-23 10:00:00' 

order by meeting_timestamp desc """, con=engine)
df.head(30)

,index,meeting_timestamp,host,host_email,invitees,invitee_emails,meeting_subject
0,58,2022-04-04 11:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Lisa Kennedy,lisa.kennedy@lockheedmartincorporation.com,Lockheed Martin - Updated ESG Capabilities Pre...
1,40,2021-04-06 13:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Lisa Kennedy,lisa.kennedy@lockheedmartincorporation.com,Lockheed Martin - ESG Capabilities Presentation
2,12,2019-03-05 13:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Lisa Kennedy,lisa.kennedy@lockheedmartincorporation.com,Lockheed Martin - LDI Strategy Presentation


In [ ]:
query_str = f"""
    SELECT  *
    FROM  meeting_data
    WHERE  invitee_emails = 'adam.clay@compass.com'
    and host_email = 'jane.moneypenny@bankwell.com'
    ORDER BY meeting_timestamp DESC;"""



last_meeting = pd.read_sql(query_str, con=engine)

last_meeting.head(20)

In [19]:
meeting_timestamp = '2024-03-26 11:00:00'
user_email = 'jane.moneypenny@bankwell.com'
query_str = f"""
    SELECT email_timestamp
        , to_names	
        , to_emails
        , from_name
        , from_email
        , email_subject
        , email_body
        FROM email_data
        WHERE email_timestamp < '{meeting_timestamp}'
        and to_emails = '{user_email}'
        order by email_timestamp desc
        limit 20
"""
result = pd.read_sql(query_str, con=engine)
recent_emails = list(result["email_body"])   

In [21]:
result.head()

,email_timestamp,to_names,to_emails,from_name,from_email,email_subject,email_body
0,2024-01-24 16:10:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jessica Palmer,jessica.palmer@hasbro.com,Hasbro - Post Divestiture Strategy Adjustments,"Hi jane,\n\nYes, this needs further thought. I..."
1,2024-01-21 09:17:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jessica Palmer,jessica.palmer@hasbro.com,Hasbro - Post Divestiture Strategy Adjustments,"Thanks Jane,\n\nThank you for the update. Seem..."
2,2023-12-13 10:15:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Michelle Jenkins,michelle.jenkins@intuitivesurgical.com,Re: Intuitive Surgical - Discussing FX Service...,"Hi Jane,\n\nYes, let's plan for that in Q1. Ho..."
3,2023-12-12 11:30:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jessica Palmer,jessica.palmer@hasbro.com,Re: Hasbro - Strategy Discussion for H1 2024 F...,"Thanks Jane,\n\nI will follow up if I have any..."
4,2023-12-05 11:30:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Timothy Ochoa,timothy.ochoa@hyatthotels.com,Re: Hyatt Hotels - 401(k) Advisory Services RFI,"Hi Jane,\n\nThanks for checking in. The RFI pr..."


In [6]:
user_email = "jane.moneypenny@bankwell.com"
meeting_timestamp = '2024-03-26 11:00:00'

mx_dt_df = pd.read_sql(f"""
select coalesce(as_of_date, '2023-01-23 10:00:00') as as_of_date
from (
SELECT max(p.as_of_date) as as_of_date
FROM preferences p
where p.email = '{user_email}'
and as_of_date <= '{meeting_timestamp}')a
""", con=engine)

#max_dt = mx_dt_df['as_of_date'][0]
max_dt = mx_dt_df['as_of_date'].iloc[0]
print("max date: ", max_dt)

query_str = f"""
SELECT p.as_of_date
, p.employee_first_name
, p.employee_last_name
, p.finance_detail
, p.news_detail
, p.macro_news_detail
, p.past_meeting_detail 
, p.email	
FROM preferences p
where p.email = '{user_email}' and p.as_of_date = '{max_dt}'
"""

pref_df = pd.read_sql(query_str, con=engine)
#holdings_detail = pref_df["finance_detail"][0]
#client_news_detail = pref_df["news_detail"][0]
#macro_news_detail = pref_df["macro_news_detail"][0]
#past_meeting_detail = pref_df["past_meeting_detail"][0]

holdings_detail = pref_df["finance_detail"].iloc[0]
client_news_detail = pref_df["news_detail"].iloc[0]
macro_news_detail = pref_df["macro_news_detail"].iloc[0]
past_meeting_detail = pref_df["past_meeting_detail"].iloc[0]

print("holdings_detail ", holdings_detail)
print("client_news_detail ", client_news_detail )
print("macro_news_detail " , macro_news_detail )
print("past_meeting_detail ", past_meeting_detail)

max date:  2024-03-10 09:00:00
holdings_detail  full
client_news_detail  none
macro_news_detail  full
past_meeting_detail  full
